In [1]:
import os
import re
import csv
import pandas as pd

In [2]:
def pripravi_imenik(ime_datoteke):
    imenik = os.path.dirname(ime_datoteke)
    if imenik:
        os.makedirs(imenik, exist_ok=True)

def zapisi_csv(slovarji, imena_polj, ime_datoteke):
    pripravi_imenik(ime_datoteke)
    with open(ime_datoteke, 'w', encoding='utf-8') as csv_datoteka:
        writer = csv.DictWriter(csv_datoteka, fieldnames=imena_polj)
        writer.writeheader()
        writer.writerows(slovarji)

def vsebina_datoteke(ime_datoteke):
    with open(ime_datoteke, encoding='utf-8') as datoteka:
        return datoteka.read()

indeksi_po_letih = [57, 5, 8, 8, 8, 9, 11, 12, 11, 13, 11, 12, 12, 13, 15, 14, 22,
18, 21, 28, 31, 36, 36, 34, 40, 36, 36, 36, 32, 31, 41, 53, 62, 57, 67, 62, 82, 
102, 113, 106, 109, 99, 105, 131, 140, 140, 165, 173, 172, 189, 188, 184, 197, 118]

In [90]:
vzorec_bloka_igre = re.compile(
    r'<td colspan="6"><img src="/images/whiteball.gif">.*?<img src="/images/printer.gif"></a></td>',
    flags = re.DOTALL
)
vzorec_igre = re.compile(
    r'&nbsp;&nbsp;\[(?P<leto>.*?)\-(?P<mesec>.*?)\-(?P<dan>.*?)\]&nbsp;'
    r'&nbsp;(?P<turnir>.*?)\n.*?</td>.*?'
    r'<td style="text-align:center;"><?b?>?(?P<igralec_crni>\D*)(?P<igralec_crni_rang>..).*?</td>.*?'
    r'<td style="text-align:center;"><?b?>?(?P<igralec_beli>\D*)(?P<igralec_beli_rang>..).*?</td>.*?'
    r'<td style="text-align:center;">(?P<zmagovalec>\w*)\+?(?P<tip_zmage>.*?)</td>',
    flags = re.DOTALL)
vzorec_bloka_narodnosti = re.compile(
    r'<select name="p" size="6" style="width:200px; font-size:87.5%; font-family:serif; margin:10px;">.*?<input type="submit" value="Search">',
    flags = re.DOTALL)
vzorec_igralca = re.compile(
    r'<option value=".*?">(?P<igralec>.*?)\s\(.*?\)')

In [31]:
igre = []
for i in range(len(indeksi_po_letih)):
    leto = str(1969 + i)
    for j in range(indeksi_po_letih[i]):
        stran = vsebina_datoteke("podatki/" + leto + "/" + str(j) + ".html")
        for blok in vzorec_bloka_igre.finditer(stran):
            igre.append(vzorec_igre.search(blok.group(0)).groupdict())

In [92]:
igralci_po_narodnosti = []
stran = vsebina_datoteke("podatki/igralci_po_narodnosti.html")
for blok in vzorec_bloka_narodnosti.findall(stran):
    igralci_po_narodnosti.append(vzorec_igralca.findall(blok))

In [98]:
zapisi_csv(igre, 
["leto", "mesec", "dan", "turnir",
"igralec_crni", "igralec_crni_rang", "igralec_beli",
"igralec_beli_rang", "zmagovalec", "tip_zmage"], 
"osnovni_podatki.csv")

In [ ]:
osnovni_podatki = pd.read_csv("osnovni_podatki.csv")
osnovni_podatki

In [102]:
dict_igralci_po_narodnosti = []
drzave = ["Kitajska", "Koreja", "Japonska", "Tajvan", "Ostali", "AI"]
for i in range(len(igralci_po_narodnosti)):
    for igralec in igralci_po_narodnosti[i]:
        dict_igralci_po_narodnosti.append({"Država" : drzave[i], "Igralec" : igralec})

In [103]:
zapisi_csv(dict_igralci_po_narodnosti, 
["Država", "Igralec"], 
"podatki_o_narodnosti.csv")

In [104]:
osnovni_podatki = pd.read_csv("podatki_o_narodnosti.csv")
osnovni_podatki

,Država,Igralec
0,Kitajska,An Dongxu
1,Kitajska,Ao Liting
2,Kitajska,Bai Baoxiang
3,Kitajska,Bai Guangyuan
4,Kitajska,Bai Yulin
...,...,...
2216,AI,Golaxy
2217,AI,Google DeepMind AlphaGo
2218,AI,Leela Zero
2219,AI,PhoenixGo
